In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
"""Custom datasets implementations."""
from pathlib import Path
from typing import Dict, Literal, Optional, Tuple, Union

import cv2
import numpy as np
import torch
from torch import Tensor

from opr.datasets.augmentations import (
    DefaultCloudSetTransform,
    DefaultCloudTransform,
    DefaultImageTransform,
)
from opr.datasets.base import BaseDataset

In [3]:
from pathlib import Path

import torch
from hydra.utils import instantiate
from omegaconf import OmegaConf
from torch.optim import Adam
from torch.optim.lr_scheduler import MultiStepLR

from opr.datasets.dataloader_factory import make_dataloaders
from opr.testing import test
from opr.training import epoch_loop

In [4]:
import os
import torch
import pandas as pd
import numpy as np
from sklearn.decomposition import PCA
from sklearn.feature_extraction.text import TfidfVectorizer
from joblib import dump, load

# def train_tfidf_pca(corpus, n_components=300, base_savepath="./opr/datasets/"):
#     vectorizer = TfidfVectorizer()
#     vectorizer.fit(corpus)
#     vectorized_corpus = vectorizer.transform(corpus).toarray()

#     pca =PCA(n_components=n_components)
#     pca.fit(vectorized_corpus)
    
#     vectorizer_savepath = os.path.join(base_savepath, 'vectorizer.joblib')
#     pca_savepath = os.path.join(base_savepath, 'pca.joblib')
#     dump(vectorizer, vectorizer_savepath) 
#     dump(pca, pca_savepath) 

In [5]:
EPOCHS = 60

IMAGE_LR = 0.0001
CLOUD_LR = 0.001
FUSION_LR = 0.001
TEXT_LR = 0.001
WEIGHT_DECAY = 0.0001

SCHEDULER_GAMMA = 0.1
SCHEDULER_STEPS = [40]

DEVICE = "cuda"
BATCH_EXPANSION_TH = 0.7
CHECKPOINTS_DIR = Path("checkpoints")

In [6]:
model_config = OmegaConf.load("configs/models/text_model.yaml")
model = instantiate(model_config)

loss_cfg = OmegaConf.load("configs/losses/triplet_margin_text.yaml")
loss_fn = instantiate(loss_cfg)

dataset_cfg = OmegaConf.load("configs/datasets/phystech_campus_text.yaml")

dataloaders = make_dataloaders(
    dataset_cfg=dataset_cfg.dataset,
    batch_sampler_cfg=dataset_cfg.sampler,
    num_workers=dataset_cfg.num_workers,
)

params_list = []
if model.image_module is not None and IMAGE_LR is not None:
    params_list.append({"params": model.image_module.parameters(), "lr": IMAGE_LR})
if model.cloud_module is not None and CLOUD_LR is not None:
    params_list.append({"params": model.cloud_module.parameters(), "lr": CLOUD_LR})
if model.fusion_module is not None and FUSION_LR is not None:
    params_list.append({"params": model.fusion_module.parameters(), "lr": FUSION_LR})
if model.text_module is not None and TEXT_LR is not None:
    params_list.append({"params": model.text_module.parameters(), "lr": IMAGE_LR})
optimizer = Adam(params_list, weight_decay=WEIGHT_DECAY)
scheduler = MultiStepLR(optimizer, milestones=SCHEDULER_STEPS, gamma=SCHEDULER_GAMMA)

if not CHECKPOINTS_DIR.exists():
    CHECKPOINTS_DIR.mkdir(parents=True)

model = model.to(DEVICE)

# ==========> TRAIN LOOP:

best_recall_at_1 = 0.0

for epoch in range(EPOCHS):
    print(f"\n\n=====> Epoch {epoch+1}:")
    # TODO: resolve mypy typing here
    train_batch_size = dataloaders["train"].batch_sampler.batch_size  # type: ignore

    print("\n=> Training:\n")

    train_stats, train_rate_non_zero = epoch_loop(
        dataloader=dataloaders["train"],
        model=model,
        loss_fn=loss_fn,
        optimizer=optimizer,
        scheduler=scheduler,
        phase="train",
        device=DEVICE,
    )

    print(f"\ntrain_rate_non_zero = {train_rate_non_zero}")

    if BATCH_EXPANSION_TH is not None:
        if BATCH_EXPANSION_TH == 1.0:
            print("Batch expansion rate is set to every epoch. Increasing batch size.")
            # TODO: resolve mypy typing here
            dataloaders["train"].batch_sampler.expand_batch()  # type: ignore
        elif train_rate_non_zero is None:
            print(
                "\nWARNING: 'BATCH_EXPANSION_TH' was set, but 'train_rate_non_zero' is None. ",
                "The batch size was not expanded.",
            )
        elif train_rate_non_zero < BATCH_EXPANSION_TH:
            print(
                "Average non-zero triplet ratio is less than threshold: ",
                f"{train_rate_non_zero} < {BATCH_EXPANSION_TH}",
            )
            # TODO: resolve mypy typing here
            dataloaders["train"].batch_sampler.expand_batch()  # type: ignore

    print("\n=> Validating:\n")

    val_stats, val_rate_non_zero = epoch_loop(
        dataloader=dataloaders["val"],
        model=model,
        loss_fn=loss_fn,
        optimizer=optimizer,
        phase="val",
        device=DEVICE,
    )

    print(f"\nval_rate_non_zero = {val_rate_non_zero}")

    print("\n=> Testing:\n")

    recall_at_n, recall_at_one_percent, mean_top1_distance = test(
        model=model,
        descriptor_key="text",
        dataloader=dataloaders["test"],
        device=DEVICE,
    )

    stats_dict = {}
    stats_dict["test"] = {
        "mean_top1_distance": mean_top1_distance,
        "recall_at_1%": recall_at_one_percent,
        "recall_at_1": recall_at_n[0],
        "recall_at_3": recall_at_n[2],
        "recall_at_5": recall_at_n[4],
        "recall_at_10": recall_at_n[9],
    }
    stats_dict["train"] = train_stats
    stats_dict["train"]["batch_size"] = train_batch_size
    stats_dict["val"] = val_stats

    # saving checkpoints
    checkpoint_dict = {
        "epoch": epoch + 1,
        "stats_dict": stats_dict,
        "model_state_dict": model.state_dict(),
        "optimizer_state_dict": optimizer.state_dict(),
    }
    torch.save(checkpoint_dict, CHECKPOINTS_DIR / f"epoch_{epoch+1}.pth")
    if recall_at_n[0] > best_recall_at_1:
        print("Recall@1 improved!")
        torch.save(checkpoint_dict, CHECKPOINTS_DIR / "best.pth")
        best_recall_at_1 = recall_at_n[0]



=====> Epoch 1:

=> Training:



train: 100%|██████████| 90/90 [00:04<00:00, 18.28it/s]


train_rate_non_zero = 1.0

=> Validating:




val: 100%|██████████| 33/33 [00:01<00:00, 19.35it/s]



val_rate_non_zero = 1.0

=> Testing:



...


train_rate_non_zero = 1.0

=> Validating:




val: 100%|██████████| 33/33 [00:02<00:00, 12.21it/s]



val_rate_non_zero = 1.0

=> Testing:



Calculating metrics: 100%|██████████| 6/6 [00:00<00:00, 16.58it/s]

Mean Recall@N:
[0.23771253 0.35023096 0.42548555 0.48398431 0.52896547 0.57143206
 0.60441905 0.63230524 0.65640937 0.68536773 0.70754954 0.72404526
 0.73975164 0.75465432 0.77141978 0.78545384 0.80008552 0.80981836
 0.82034282 0.82819541 0.83548352 0.84088665 0.84900046 0.85549318
 0.8665562 ]
Mean Recall@1% = 0.5714320572158791
Mean top-1 distance = 0.02942885007110731


In [7]:
from torch.utils.data import DataLoader
from opr.datasets.dataloader_factory import make_collate_fn

real_val_dataset = instantiate(dataset_cfg.dataset, subset="val")

num_workers = 4
batch_size = 16

real_test_dataloader = DataLoader(
    dataset=real_val_dataset,
    batch_size=batch_size,
    collate_fn=make_collate_fn(real_val_dataset, batch_split_size=None),
    num_workers=num_workers,
    pin_memory=False,
    drop_last=False,
)

In [8]:
real_recall_at_n, real_recall_at_one_percent, real_mean_top1_distance = test(
    model=model,
    descriptor_key="text",
    dataloader=real_test_dataloader,
    device=DEVICE,
    dist_thresh=25.0
)

Calculating metrics: 100%|██████████| 6/6 [00:00<00:00, 138.52it/s]

Mean Recall@N:
[0.18526102 0.31069173 0.40750433 0.48487334 0.54178005 0.59866342
 0.64616159 0.68246205 0.72151508 0.75011433 0.77330197 0.796352
 0.81429287 0.82703898 0.84407971 0.85914799 0.87407275 0.88735718
 0.89854956 0.90782049 0.91186266 0.9190182  0.92750961 0.93862706
 0.94358695]
Mean Recall@1% = 0.31069173012814627
Mean top-1 distance = 0.030758283096002065


In [11]:
real_recall_at_n, real_recall_at_one_percent, real_mean_top1_distance = test(
    model=model,
    descriptor_key="text",
    dataloader=real_test_dataloader,
    device=DEVICE,
    dist_thresh=5.0
)

Calculating metrics: 100%|██████████| 6/6 [00:00<00:00, 148.63it/s]

Mean Recall@N:
[0.05004573 0.08028988 0.10914735 0.1426537  0.16656167 0.1883161
 0.20587251 0.22635698 0.24919296 0.27811522 0.29664631 0.31208683
 0.33288934 0.34660125 0.36433831 0.38202838 0.39425828 0.41383285
 0.42958689 0.43366448 0.44739285 0.45882668 0.4702329  0.48141552
 0.4957535 ]
Mean Recall@1% = 0.08028988419741347
Mean top-1 distance = 0.02984082018928332


In [9]:
recall_at_n, recall_at_one_percent, mean_top1_distance = test(
    model=model,
    descriptor_key="text",
    dataloader=dataloaders["test"],
    device=DEVICE,
)

Calculating metrics: 100%|██████████| 6/6 [00:00<00:00, 16.87it/s]

Mean Recall@N:
[0.23771253 0.35023096 0.42548555 0.48398431 0.52896547 0.57143206
 0.60441905 0.63230524 0.65640937 0.68536773 0.70754954 0.72404526
 0.73975164 0.75465432 0.77141978 0.78545384 0.80008552 0.80981836
 0.82034282 0.82819541 0.83548352 0.84088665 0.84900046 0.85549318
 0.8665562 ]
Mean Recall@1% = 0.5714320572158791
Mean top-1 distance = 0.02942885007110731


In [10]:
recall_at_n_5, recall_at_one_percent_5, mean_top1_distance_5 = test(
    model=model,
    descriptor_key="text",
    dataloader=dataloaders["test"],
    device=DEVICE,
    dist_thresh=5.0
)

Calculating metrics: 100%|██████████| 6/6 [00:00<00:00, 16.42it/s]

Mean Recall@N:
[0.11585745 0.17756552 0.22503744 0.25630179 0.28478687 0.31294511
 0.33777806 0.35835067 0.3781953  0.39319692 0.40845309 0.42487215
 0.43689295 0.44945015 0.46307803 0.47667892 0.48783226 0.5001431
 0.50902053 0.52183219 0.53129541 0.53883277 0.54832413 0.55783808
 0.56642465]
Mean Recall@1% = 0.3129451101029596
Mean top-1 distance = 0.029024499735848076
